In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ishas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df = pd.read_csv('fakenews.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
df.shape

(20800, 5)

In [12]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [13]:
df = df.fillna('')

In [15]:
df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

Using title and author for prediction

In [19]:
df['features'] = df['title'] + ' ' + df['author']

In [20]:
df['features'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart Daniel J. Flynn'

In [23]:
X = df.drop('label',axis=1)
y = df['label']

### Data Preprocessing

In [28]:
ps = PorterStemmer()

In [29]:
def preprocessing(text):
    cleaned_text = re.sub('[^A-Za-z]',' ',text)
    cleaned_text = cleaned_text.lower()
    cleaned_text = cleaned_text.split()
    cleaned_text = [ps.stem(word) for word in cleaned_text if not word in stopwords.words('english')]
    cleaned_text = ' '.join(cleaned_text)
    return cleaned_text

In [30]:
df['features'] = df['features'].apply(preprocessing)

In [32]:
df['features'].head()

0    hous dem aid even see comey letter jason chaff...
1    flynn hillari clinton big woman campu breitbar...
2               truth might get fire consortiumnew com
3    civilian kill singl us airstrik identifi jessi...
4    iranian woman jail fiction unpublish stori wom...
Name: features, dtype: object

In [33]:
X = df['features'].values
y = df['label']

In [34]:
from sklearn.model_selection import train_test_split

In [36]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [40]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [41]:
preds = model.predict(X_test)

In [45]:
print('Accuracy Score',accuracy_score(y_test,preds))

Accuracy Score 0.9723193473193473


In [49]:
##Building a prediction system
X_new = X_test[0]
prediction = model.predict(X_new)
if prediction[0] == 0:
    print('Real News')
else:
    print('Fake news')

Fake news
